In [200]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [201]:
onboarding_data = pd.read_csv('data.csv')
onboarding_data.columns

Index(['SUBMISSION_DATE', 'ORIGINATOR', 'EMPLOYEE', 'REHIRE',
       'CURRENT_OR_FORMER_STUDENT', 'DEPARTMENT', 'SALARY_ADMIN_PLAN',
       'BIO_UD_ID', 'MY_UD_SETTINGS', 'PERSONAL_CONTACT', 'EDD', 'W4',
       'DIRECT_DEPOSIT_FORM', 'BAYH_DOLE', 'JED', 'CBC', 'I9'],
      dtype='object')

In [202]:
from plotly.subplots import make_subplots
import math

onboarding_q1 = onboarding_data.copy()

form_columns = [['EDD', 'W4', 'DIRECT_DEPOSIT_FORM', 'BAYH_DOLE'], ['JED', 'CBC', 'I9']]

fig = make_subplots(rows=2, cols=4, specs=[[{ 'type': 'bar' },{ 'type': 'bar' },{ 'type': 'bar' },{ 'type': 'bar' }],[{ 'type': 'bar' },{ 'type': 'bar' },{ 'type': 'bar' },{ 'type': 'bar' }]])

for row_ind, each_row in enumerate(form_columns):
    for element_ind, each_element in enumerate(each_row):
        onboarding_q1[f'{each_element}_is_complete'] = onboarding_q1[each_element].apply(lambda x: False if type(x) is float and math.isnan(x) else True)
        is_completed = len(onboarding_q1[onboarding_q1[f'{each_element}_is_complete'] == True])
        is_not_completed = len(onboarding_q1[onboarding_q1[f'{each_element}_is_complete'] == False])
        stat_bar_chart = go.Bar(x=['Is Complete', 'Is Not Completed'], y=[is_completed, is_not_completed], text=f'{each_element.upper()} Analysis')
        fig.add_trace(stat_bar_chart, row=row_ind + 1, col=element_ind + 1)

fig.update_layout(title_text="Statistical Reporting | Form Submissions", height=850, template="plotly_dark")
fig.show()

In [203]:
import ipywidgets as widgets
from datetime import datetime
from IPython.display import display, clear_output


q2_data = onboarding_q1.copy()

today = datetime.now()
q2_data['SUBMISSION_DATE'] = pd.to_datetime(q2_data['SUBMISSION_DATE'], format='%Y-%m-%dT%H:%M:%S')
q2_data['SUBMISSION_DAYS_SINCE'] = q2_data['SUBMISSION_DATE'].apply(lambda x: (pd.Timestamp(year=today.year, month=today.month, day=today.day, hour=today.hour, minute=today.minute, second=today.second) - x).days)

num_slider = widgets.IntSlider(value=0, min=0, max=np.max(q2_data['SUBMISSION_DAYS_SINCE']), step=1, disabled=False, description='Submissions up to X days ago:', readout=True, readout_format='d', orientation='horizontal')

output = widgets.Output()

def update_slider(change):
    new_change = change['new']
    if type(new_change) is dict and 'value' in new_change:
        value = new_change['value']
        found_submissions = q2_data[q2_data['SUBMISSION_DAYS_SINCE'] >= value]['ORIGINATOR'].tolist()
        with output:
            output.clear_output()
            for each_submission in found_submissions:
                print(each_submission)
    elif type(new_change) is int:
        found_submissions = q2_data[q2_data['SUBMISSION_DAYS_SINCE'] >= new_change]['ORIGINATOR'].tolist()
        with output:
            output.clear_output()
            for each_submission in found_submissions:
                print(each_submission)


num_slider.observe(update_slider)
container = widgets.VBox([num_slider, output])
container.layout.height = '200px'

display(container)

In [204]:
q3_data = q2_data.copy()

total_form_columns = len(form_columns[0]) + len(form_columns[1])
specs = [[{'type': 'bar'}]] * total_form_columns
not_completed_charts = make_subplots(rows=total_form_columns, cols=1, specs=specs)

for each_form_col_ind, each_form_col in enumerate(form_columns[0] + form_columns[1]):
    is_completed = len(q3_data[q3_data[f'{each_form_col}_is_complete'] == True])
    is_not_completed = len(q3_data[q3_data[f'{each_form_col}_is_complete'] == False])
    not_completed_charts.add_trace(go.Bar(x=['Is Completed', 'Not Completed'], y=[is_completed, is_not_completed], text=each_form_col), row=each_form_col_ind + 1, col=1)

not_completed_charts.update_layout(height=1000, title_text='Amount of forms completed vs not completed', template='plotly_dark')
not_completed_charts.show()

In [225]:
q4_data = q3_data.copy()
completed_locs = set()

form_columns_flattened = form_columns[0] + form_columns[1]

for each_col in form_columns_flattened:
    found_people = q4_data[q4_data[f'{each_col}_is_complete'] == True]
    completed_locs.update(set(list(found_people.index)) - completed_locs)

completed_tasks = q4_data.loc[list(completed_locs)]

completed_tasks.to_csv('completed_tasks.csv')